# Leveraging Mean/Target Encoding for Better Feature Engineering and Better Predictions

The zip (postal) code you live in says a tremendous amount about you.  (At least in North America). Where you live suggests: your annual income, whether you have children, the TV shows you watch and your political leanings. 

In the United States there are over 41,000 unique zip codes.  Zip codes are largely categorical.  There is some broad meaning in the first two digits of the zip code.  For example, Hawaii zip codes start with 9 and Maine zip codes start with 0.  Beyond, very general geographic information, the codes themselves really provide little value. 

What I if said that I live in 76092?   Does that really tell you much about me?   

Not really.

Now, if I googled that zip code.  I would find out that 76092 is Southlake, Texas.  Southlake is one of the most affluent areas of Texas.  Living in 76092 means that I probably make more than 150k a year and have a college education. 

My point here is that the code itself doesn't tell us much.  We have to find creative ways to extract value from the zip code.

So, given that zip codes can tell us a great deal about the people who live in them, how do we extract this information and exploit it in a machine learning model?


One way to deal with categorical variables like zip codes is to split them into dummy variables.  This is what "One Hot Encoder" does.  When you have a categorical variable with 41,000 unique values, dummy variables really won't be much help in my experience.    Using "One Hot Encoder" on zip code means you'll create 40,999 new independent variables.  Jeez, what a mess!


Another approach is something called Mean/Target Encoding.  In full disclosure, I have used this technique for twenty-five years and I had never heard anyone call it Mean/Target Encoding until very recently.  Whatever you want to call it, it works quite well for categorical variables like zip code, NAICS, census block or any other meaningful categorical variable that has many distinct values.

In this example, I walk through a customer prospecting use case.  The company is looking to grow its customer base but has limited marketing/sales resources.  Because they don't have enough money to contact everyone in the database, they will use a predictive model to predict those prospects most likely to buy acquire their product. I will not be building a model in this notebook.  Rather, I will show you can leverage your historical data and zip code to create features that will build more predictive machine learning models.

As I write this, it is February 2020.  So, the goal is to build a model that will predict February 2020.  To do this, we will use data from 2019.  As we do our feature engineering, it is best not to use the same data you use for your model.  Doing so leads to some major causality issues and over fitting.  Instead of 2019, I will be using data from 2018 to build my features.  So, just to re-cap.  I will use 2018 data to build my features.  Use 2019 data to build a model and apply that model to current prospects in 2020.

What if you don't have multiple years of data?  In that case, I would recommend creating a separate sample of the data set to build your features.  So, as you build your model, you would split your data into four groups.  This would include a Training, a Testing, a Validation and a Feature Building data set.

And finally, I will be creating summaries that are specific to a customer acquisition problem, but this technique applies to pretty much everything.  For example, you could create average costs for supplier codes in health care.  Or, Out Of Business rates for certain NAICS codes.  It is important to understand the process and realize that this technique can be applied to many different situations.


## Table of contents

1. [Getting Setup](#setup)<br>
    1.1 [Install Relevant Libraries](#installlibs)<br>
    1.2 [Import Relevant Libraries](#import)<br>
    1.3 [Load data from GitHub](#pull)<br>
2. [Explore Data](#explore)<br>

3. [Build Features](#build)<br>
 
4. [Append Features to the Modeling Data Set](#append)<br>


## 1.0 Getting Setup <a id="setup"></a>

#### 1.1 Install all of the relevant Python Libraries <a id="installlibs"></a>

In [1]:
!pip install --upgrade numpy 

     |████████████████████████████████| 14.5MB 10.3MB/s eta 0:00:01
ERROR: tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
ERROR: autoai-libs 1.10.5 has requirement pandas>=0.24.2, but you'll have pandas 0.24.1 which is incompatible.
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4


 #### 1.2 Import relevant libraries <a id="import"></a>

In [2]:


import pandas as pd


import numpy as np
import numpy.dual as dual



#Un-Comment these options if you want to exapand the number of rows and columns of you see visually in the notebook.
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

####  1.3 Pull Data from GitHub  <a id="pull"></a>

In [3]:
!rm YEAR_2018_1.csv
!wget https://raw.githubusercontent.com/shadgriffin/zip_code/master/YEAR_2018_1.csv

rm: cannot remove ‘YEAR_2018_1.csv’: No such file or directory
--2020-08-18 13:00:14--  https://raw.githubusercontent.com/shadgriffin/zip_code/master/YEAR_2018_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12874808 (12M) [text/plain]
Saving to: ‘YEAR_2018_1.csv’

100%[======================================>] 12,874,808  --.-K/s   in 0.1s    

2020-08-18 13:00:15 (126 MB/s) - ‘YEAR_2018_1.csv’ saved [12874808/12874808]



In [4]:
!rm YEAR_2018_2.csv
!wget https://raw.githubusercontent.com/shadgriffin/zip_code/master/YEAR_2018_2.csv

rm: cannot remove ‘YEAR_2018_2.csv’: No such file or directory
--2020-08-18 13:00:18--  https://raw.githubusercontent.com/shadgriffin/zip_code/master/YEAR_2018_2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11339381 (11M) [text/plain]
Saving to: ‘YEAR_2018_2.csv’

100%[======================================>] 11,339,381  --.-K/s   in 0.1s    

2020-08-18 13:00:19 (107 MB/s) - ‘YEAR_2018_2.csv’ saved [11339381/11339381]



As I mentioned earlier, we hope to build a model on data from 2019 to predict customer acquisition in 2020. To build our features, we will use data from 2018.

Pull in the 2018 data from Github

In [5]:
pd_datax = pd.read_csv("YEAR_2018_1.csv")
pd_datay = pd.read_csv("YEAR_2018_2.csv")

df_data_1 = pd.concat([pd_datax,pd_datay])

## 2.0 Data Exporation <a id="explore"></a>

In [6]:
df_data_1.head()

,ZIP5,HAS_CHILDREN,PRIMARY_LANGUAGE,INCOME,AGE_IN_YEARS,LENGTH_OF_RESIDENCE,HOME_OWNER_RENTER,CUSTOMER,CHURN,ACQ,REVENUE,YEAR
0,76248.0,1.0,English,125000.0,43.901437,2.0,O,1,0,0,52,2018
1,75007.0,1.0,English,87500.0,51.816564,3.0,O,1,0,0,100,2018
2,75071.0,1.0,English,57500.0,54.231348,7.0,O,1,0,1,240,2018
3,75089.0,1.0,Other,87500.0,39.734428,9.0,O,1,0,0,72,2018
4,76110.0,1.0,Spanish,52500.0,51.063655,12.0,O,1,0,0,172,2018


This is a fairly simple data set.  Here is a description of the fields.

ZIP5 -- The zip code of the individual.

HAS_CHILDREN -- 1 means the individual has children.  0 means they do not.

PRIMARY_LANGUAGE -- The primary language of the individual.

INCOME -- Income of the individual.

AGE_IN_YEARS -- Age of the individual.

LENGTH_OF_RESIDENCE -- Length of time the individual has resided at their current address.

HOME_OWNER_RENTER - O means the individual owns their home.  

CUSTOMER - indicates if the record belongs to a customer. 1 is a customer 0 is a non-customer.

CHURN -- 1 means the individual canceled their product in 2018.  0 Means they did not cancel.

ACQ -- 1 means the individaul acquired the product in 2018.

REVENUE -- Total Revenue of the Individual in 2018.

YEAR -- the YEAR of the individual record.



It may be obvious, but the last five fields are the important ones from a feature engineering perspective.  In the next few lines of code, for each zip code we will derive the following.

ZIP_PENETRATION_RATE -- The percentage of individuals in a zip code who are customers.

ZIP_CHURN_RATE -- the churn rate of a specific zip_code

ZIP_AQC_RATE -- The customer acquisition rate of the zip code.

ZIP_AVG_REV  -- the average revenue for customers in specific zip code.

ZIP_MEDIAN_REV -- the median revenue for customers in a specific zip code

ZIP_CUSTOMERS -- The total number of customers in a specific zip code.

ZIP_POPULATION -- The total number of Individuals in a specific zip code.

ZIP_CHURNERS -- The total number of Churners in a specific zip code.

ZIP _REVENUE -- The total revenue in a specific zip code.

By creating these new fields, we can extract the value imbeded in zip code. 


## 3.0 Build Features <a id="build"></a>

In [7]:
dfx=df_data_1

In [8]:

# Create churn features
zip_churn = pd.DataFrame(dfx.groupby(['ZIP5'])['CHURN'].agg(['sum']))
zip_churn['TOTAL']=dfx.CHURN.sum()

zip_churn.columns = ['ZIP_CHURNERS','TOTAL_CHURNERS']


#Create customer and popluation features
zip_cust = pd.DataFrame(dfx.groupby(['ZIP5'])['CUSTOMER'].agg(['sum','count']))
zip_cust['TOTAL_CUSTOMERS']=dfx.CUSTOMER.sum()
zip_cust['TOTAL_POPULATION']=dfx.CUSTOMER.sum()

zip_cust.columns = ['ZIP_CUSTOMERS','ZIP_POPULATION','TOTAL_CUSTOMERS','TOTAL_POPULATION']

#create acquisition features
zip_acq = pd.DataFrame(dfx.groupby(['ZIP5'])['ACQ'].agg(['sum']))

zip_acq['TOTAL']=dfx.ACQ.sum()

zip_acq.columns = ['ZIP_ACQUISITIONS','TOTAL_ACQUISITIONS']


#Create Total Revenue Features
zip_rev = pd.DataFrame(dfx.groupby(['ZIP5'])['REVENUE'].agg(['sum']))
zip_rev['TOTAL']=dfx.REVENUE.sum()

zip_rev.columns = ['ZIP_REVENUE','TOTAL_REVENUE']

#create median revenue features.
df_cust=dfx[dfx['CUSTOMER']==1]

zip_med_rev = pd.DataFrame(df_cust.groupby(['ZIP5'])['REVENUE'].agg(['median']))
zip_med_rev['TOTAL']=df_cust.REVENUE.median()

zip_med_rev.columns = ['MED_REVENUE','TOTAL_MED_REVENUE']


Append the features into a single data frame.

In [9]:
df_18 = pd.concat([zip_cust,zip_acq, zip_churn, zip_rev,zip_med_rev], axis=1)
df_18.reset_index(level=0, inplace=True)


Create additional features from the existing features.

Note that you have to be careful of small sample sizes  when calculating averages and ratios. In this example, I only calculate a rate or average for a zip code if there are more than 100 people in the zip code.  You want to avoid situations where a metric is high or low only because the sample is small.  For example, if you have 2 people in a zip code and one is a customer, the penetration rate would be extremely high (50%).  This high number doesn't mean that the zip code is fertile grounds for prospecting.   Maybe it is.  Maybe it isn't.  If you only have two people in your sample, the statistic really has no value.

Like I mentioned earlier, I only use a statistic or ratio if there are more than 100 people are in the zip code.  If there is less than 100 people, I use the global average or ratio.  Note that there is nothing magically about 100.  You should use a number that is logical and meets the needs of your business case.  I have also seen examples were people will use a weighted metric if there is a small sample size for a particular group. That is, they will take the cases in the zip code and combine with the global average/ratio in a weighted manner.  I think that is a bit over-kill, but if you floats your boat, go for it.



In [10]:
df_18['ZIP_PENETRATION_RATE'] = np.where(((df_18['ZIP_CUSTOMERS'] <100 )), (df_18['TOTAL_CUSTOMERS'])/(df_18['TOTAL_POPULATION']), (df_18['ZIP_CUSTOMERS'])/(df_18['ZIP_POPULATION']))
df_18['ZIP_ACQ_RATE'] = np.where(((df_18['ZIP_CUSTOMERS'] <100 )), (df_18['TOTAL_ACQUISITIONS'])/(df_18['TOTAL_POPULATION']), (df_18['ZIP_ACQUISITIONS'])/(df_18['ZIP_POPULATION']))
df_18['ZIP_CHURN_RATE'] = np.where(((df_18['ZIP_CUSTOMERS'] <100 )), (df_18['TOTAL_CHURNERS'])/(df_18['TOTAL_CUSTOMERS']), (df_18['ZIP_CHURNERS'])/(df_18['ZIP_CUSTOMERS']))
df_18['ZIP_AVG_REV'] = np.where(((df_18['ZIP_CUSTOMERS'] <100 )), (df_18['TOTAL_REVENUE'])/(df_18['TOTAL_CUSTOMERS']), (df_18['ZIP_REVENUE'])/(df_18['ZIP_CUSTOMERS']))
df_18['ZIP_MED_REV'] = np.where(((df_18['ZIP_CUSTOMERS'] <100 )), (df_18['TOTAL_MED_REVENUE']), (df_18['MED_REVENUE']))

df_18=df_18[['ZIP5', 'ZIP_CUSTOMERS', 'ZIP_POPULATION', 'ZIP_ACQUISITIONS', 
       'ZIP_CHURNERS', 'ZIP_REVENUE','ZIP_PENETRATION_RATE',
       'ZIP_ACQ_RATE', 'ZIP_CHURN_RATE', 'ZIP_AVG_REV', 'ZIP_MED_REV']]

df_18.head()

,ZIP5,ZIP_CUSTOMERS,ZIP_POPULATION,ZIP_ACQUISITIONS,ZIP_CHURNERS,ZIP_REVENUE,ZIP_PENETRATION_RATE,ZIP_ACQ_RATE,ZIP_CHURN_RATE,ZIP_AVG_REV,ZIP_MED_REV
0,75001.0,231,886,62,30,40181,0.260722,0.069977,0.129870,173.943723,176.0
1,75002.0,1074,4636,295,182,179725,0.231665,0.063632,0.169460,167.341713,162.0
2,75006.0,772,2950,182,107,130349,0.261695,0.061695,0.138601,168.845855,168.0
3,75007.0,1024,4027,249,142,174408,0.254284,0.061833,0.138672,170.320312,166.5
4,75009.0,149,569,28,16,25281,0.261863,0.049209,0.107383,169.671141,165.0


It is easy to get lost in python code, but let's take a step back and remember our objective.  Our objective is to extract the hidden value inside a categorical variable with 41000 unique values.  That is, make a variable useful when the actual values of the variable are not useful.  That is what we have done.  For example, the value 75001 is not very useful.  Knowing, however, that the zip code 75001 has a product penetration rate of .260722 is very useful. 

## 4.0 Append Features to the Modeling Data Set <a id="append"></a>

Now that we have created out zip code level features, we can append them to our modeling data set, 2019 data.

Collect the data from GitHub

In [11]:

!rm YEAR_2019.csv
!wget https://raw.githubusercontent.com/shadgriffin/zip_code/master/YEAR_2019.csv
df_2019 = pd.read_csv("YEAR_2019.csv")

--2020-08-18 13:01:33--  https://raw.githubusercontent.com/shadgriffin/zip_code/master/YEAR_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18917414 (18M) [text/plain]
Saving to: ‘YEAR_2019.csv’

100%[======================================>] 18,917,414  --.-K/s   in 0.1s    

2020-08-18 13:01:33 (122 MB/s) - ‘YEAR_2019.csv’ saved [18917414/18917414]



In [12]:
df_2019 = pd.merge(df_2019, df_18, how='inner', on=['ZIP5'])
df_2019.shape

(302012, 21)

In [13]:
df_2019.head()

,ZIP5,ZIP4,HAS_CHILDREN,PRIMARY_LANGUAGE,INCOME,AGE_IN_YEARS,LENGTH_OF_RESIDENCE,HOME_OWNER_RENTER,ACQ,CUSTOMER,...,ZIP_CUSTOMERS,ZIP_POPULATION,ZIP_ACQUISITIONS,ZIP_CHURNERS,ZIP_REVENUE,ZIP_PENETRATION_RATE,ZIP_ACQ_RATE,ZIP_CHURN_RATE,ZIP_AVG_REV,ZIP_MED_REV
0,75001.0,3453.0,1.0,English,87500.0,61.902806,2.0,O,0,0,...,231,886,62,30,40181,0.260722,0.069977,0.12987,173.943723,176.0
1,75001.0,3107.0,1.0,English,125000.0,61.232033,11.0,O,0,0,...,231,886,62,30,40181,0.260722,0.069977,0.12987,173.943723,176.0
2,75001.0,5151.0,1.0,English,87500.0,52.736482,15.0,O,0,0,...,231,886,62,30,40181,0.260722,0.069977,0.12987,173.943723,176.0
3,75001.0,6075.0,0.0,English,12500.0,35.397673,1.0,O,0,0,...,231,886,62,30,40181,0.260722,0.069977,0.12987,173.943723,176.0
4,75001.0,4942.0,0.0,English,125000.0,72.150582,6.0,O,0,0,...,231,886,62,30,40181,0.260722,0.069977,0.12987,173.943723,176.0


Now we can use our features to build an Acquisition Model using ACQ as a dependent variable.  The features should allow us to extract the full predictive value from the zip code field.

One last note.  When you actually deploy the model to our 2020 data, use the 2019 data to create your zip code feature variables, not 2018.   This makes sense right?  The features are extracted from the most recent year of data.  When we build a model for 2019, this was 2018.   When we deploy the model for 2020, this would be 2019.

### Author

**Shad Griffin**, is a Data Scientist at the IBM Global Solution Center in Dallas, Texas

#### Copyright for Watson Studio community

<hr>
Copyright &copy; IBM Corp. 2019. This notebook and its source code are released under the terms of the MIT License.